<img src="https://files.cdn.thinkific.com/file_uploads/133292/images/d57/1b0/1a9/P03.jpg">

`MCSDSC02P03V3______`

`MCSDSELIMC02P03V2250621`

`MCSDSELIMC02P03V1130621`

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Отличная работа! Проект принят.
Несколько комментариев ниже

## Постановка задачи

Провести анализ данных страховой компании:
* Проанализировать страховые случае, возраст клиента и доход
* Выяснить как проанализированные данные влияют статус выплаты страховой компенсации

## Импорт данных и библиотек

Импортируем библиотеки, прочитаем файл `insurance.csv` и получим общую информацию о данных.

In [232]:
import pandas as pd
user_data = pd.read_csv('../../data/insurance.csv')
user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2301 entries, 0 to 2300
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              2301 non-null   int64  
 1   perc_of_compensation_paid       2301 non-null   float64
 2   age_in_days                     2301 non-null   float64
 3   age_id                          2301 non-null   int64  
 4   age_group                       2301 non-null   object 
 5   income                          2236 non-null   float64
 6   count_3-6_months_late           2301 non-null   int64  
 7   count_6-12_months_late          2301 non-null   int64  
 8   count_more_than_12_months_late  2301 non-null   int64  
 9   cause                           2301 non-null   object 
dtypes: float64(3), int64(5), object(2)
memory usage: 179.9+ KB


Столбцы содержат следующую информацию:

* id - ID клиента
* perc_of_compensation_paid - процент выплачиваемой компенсации
* age_in_days - возраст клиента в днях
* age_id - ID категории возраста клиента
* age_group - категория возраста клиента
* income - доход клиента
* count_3-6_mounth_late - полис оформлен не позднее 3-6 месяцев
* count_6-12_mounth_late - полис оформлен не позднее 6-12 месяцев
* count_more_than_12_mounth_late - полис оформлен больше года назад
* couse - причина страховой выплаты

### Выводы

* Данные содержат информацию о держателе страхового полиса и и информацию о самом старховом полисе.
* Датасет с данными состоит из 10 столбцов и 2301 строк.
* Датасет содержит 3 ненулевых столбца типа float64, 5 ненулевых столбца типа int64, 2 ненулевых столбца типа object.

## Предобработка данных

**Обработка пропусков**

Проверим данные на наличие пропусков, просуммировав их.

In [233]:
user_data.isnull().sum()
#user_data.isna().sum()

id                                 0
perc_of_compensation_paid          0
age_in_days                        0
age_id                             0
age_group                          0
income                            65
count_3-6_months_late              0
count_6-12_months_late             0
count_more_than_12_months_late     0
cause                              0
dtype: int64

Заполним пропуски в столбце `income`.

In [174]:
#user_data[user_data.age_id == 2][['age_group','income']]
#user_data.query('age_id == 2')[['age_group','income']]
#user_data.loc[user_data.age_id == 2, ['age_group', 'income']]

In [234]:
#user_data.loc[user_data.age_id == 2, 'income'].mean()
user_data.loc[user_data.age_id == 2, 'income'] = user_data.loc[user_data.age_id == 2, 'income'].fillna(user_data.loc[user_data.age_id == 2, 'income'].mean())
user_data.loc[user_data.age_id == 3, 'income'] = user_data.loc[user_data.age_id == 3, 'income'].fillna(user_data.loc[user_data.age_id == 3, 'income'].mean())
user_data.loc[user_data.age_id == 4, 'income'] = user_data.loc[user_data.age_id == 4, 'income'].fillna(user_data.loc[user_data.age_id == 4, 'income'].mean())
    

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Вариант рабочий, но можно попроще)

```insur_data=insur_data.fillna(insur_data.groupby(['age_id']).transform("median"))```

In [173]:
#user_data.loc[user_data.age_id == 2].isnull().sum() #11
#user_data.loc[user_data.age_id == 3].isnull().sum() #23
#user_data.loc[user_data.age_id == 4].isnull().sum() #31

Проверяем, что пропусков в `income` не осталось.

In [235]:
user_data.income.isnull().sum()

0

**Замена типа данных**

Проверяем, что в столбцах `age_in_days` и `income` вещественный тип данных.

In [236]:
user_data[['age_in_days','income']].dtypes

age_in_days    float64
income         float64
dtype: object

Для этих столбцов нужен целочисленный тип, поэтому переведем их, а также избавимся от отрицательных значений в `income`.

In [237]:
user_data.income = user_data.income.astype('int64').abs()

In [239]:
user_data.age_in_days = user_data.age_in_days.astype('int64').abs()

In [241]:
# проверка
user_data.loc[:,('age_in_days','income')].dtypes

age_in_days    int64
income         int64
dtype: object

In [262]:
# проверка 2
if not (user_data.loc[user_data.income < 0].income.count() | user_data.loc[user_data.age_in_days < 0].age_in_days.count()):
    print('all good')

all good


<h3><font color="#89DD00">Комментарий ассистента</font></h3>

С проверкой крутое решение!

**Поиск дубликатов с учетом регистра**

Проверим уникальные категории в столбце `age_group`.

In [263]:
user_data.age_group.unique()

array(['Пожилой', 'пожилой', 'Зрелый', 'Молодой', 'ЗРЕЛЫЙ', 'ПОЖИЛОЙ',
       'зрелый', 'молодой', 'МОЛОДОЙ'], dtype=object)

Приведем их к нижнему регистру.

In [264]:
user_data.age_group = user_data.age_group.str.lower()

**Обработка дубликатов**

Установим количество явных дубликатов. Если найдутся, то удалим и снова проверим, что их не осталось.

In [265]:
user_data.duplicated().sum()

7

In [266]:
user_data.drop_duplicates(inplace=True)
user_data.reset_index(drop=True)

,id,perc_of_compensation_paid,age_in_days,age_id,age_group,income,count_3-6_months_late,count_6-12_months_late,count_more_than_12_months_late,cause
0,649,0.001,27384,4,пожилой,195212,0,0,0,повреждение автомобиля
1,81136,0.124,23735,4,пожилой,285140,0,0,0,упало дерево на автомобиль
2,70762,1.000,17170,3,зрелый,186030,0,0,0,сломала ногу в отпуске
3,53935,0.198,16068,3,зрелый,123540,0,0,0,хищение автомобиля (каско)
4,15476,0.041,10591,2,молодой,200020,1,0,0,несчастный случай в отпуске
...,...,...,...,...,...,...,...,...,...,...
2289,16440,0.474,18628,3,зрелый,201292,0,0,0,дтп
2290,109416,0.413,17525,3,зрелый,201292,0,0,0,кража в квартире
2291,4268,0.109,21906,4,пожилой,195212,0,0,0,проникли в жилье
2292,85708,0.036,25925,4,пожилой,195212,0,0,0,обокрали дачу


In [267]:
user_data.duplicated().sum()

0

**Стэмминг**

Посчитаем уникальные значения в столбце `cause`.

In [268]:
user_data.cause.value_counts().count()

26

In [269]:
user_data.cause.value_counts()

кража автомобиля               112
травма во время отдыха         107
сломала ногу в отпуске          95
упало дерево на автомобиль      94
украли багаж в отпуске          94
травма в отпуске                94
сгорел дом                      93
замкнуло электричество дома     91
сгорела квартира                91
врезались в автомобиль          91
несчастный случай в отпуске     91
сотрясение во время отдыха      90
проникли в жилье                89
повреждение автомобиля          89
потоп в квартире                88
хищение автомобиля (каско)      88
несчастный случай на отдыхе     85
повреждение авто                85
кража авто                      82
обокрали дачу                   81
поломка авто                    81
кража в квартире                79
украли авто                     78
взломали квартиру               77
дтп                             76
затопило дом                    73
Name: cause, dtype: int64

Выделим очевидные разные стеммы: 
* авто
* отпуск
* квартира
* дом
* отдых
* дтп

Из них можно выделить следующие виды страхования: 
* страхование автомобиля
* страхование во время отпуска
* страхование имущества

Напишем функцию, которая принимает неизменяемую часть слова и возвращает все названия категорий с ним. Сначала попробуем стемминг и если не поймаем «чужих» названий — значит этот метод подойдет.

In [270]:
from nltk.stem import SnowballStemmer
russian_stemmer = SnowballStemmer('russian')

In [271]:
def get_cause_by_word(data, search_word):
    return [ cat for cat in data.cause.unique() if russian_stemmer.stem(search_word) in cat]

In [272]:
get_cause_by_word(user_data, 'отдых')

['травма во время отдыха',
 'сотрясение во время отдыха',
 'несчастный случай на отдыхе']

**Категоризация данных**

Cловарь — распространенный способ хранения информации, который помогает убрать текстовые параметры из основной таблицы и увеличить эффективность работы с данными. Выделим словарь для возрастных групп.

In [290]:
#age_group_df = user_data[['age_id', 'age_group']].drop_duplicates().reset_index(drop=True)
age_group_df = user_data.loc[:,('age_id', 'age_group')].drop_duplicates().set_index('age_id')
age_group_df

,age_group
age_id,
4,пожилой
3,зрелый
2,молодой


Категоризируем причины выплаты страховых платежей с помощью лемматизации.

In [399]:
# тут прям очень долго мучалась с перформансом, пока не вынесла инициализацию Mystem :)
m = Mystem()
def insurance_cause_category(cause_text):
    from pymystem3 import Mystem
    
    cat_1, cat_1_list = 'страхование автомобиля', ['автомобиль', 'авто', 'дтп']
    cat_2, cat_2_list = 'страхование во время отпуска', ['отдых', 'отпуск']
    cat_3, cat_3_list = 'страхование имущества', ['квартира', 'дом', 'дача', 'жилье']
    
    #m = Mystem()
    lemmas = m.lemmatize(cause_text)
    
    # categorize
    if any(el in cat_1_list for el in lemmas):
        return cat_1
    elif any(el in cat_2_list for el in lemmas):
        return cat_2
    elif any(el in cat_3_list for el in lemmas):
        return cat_3
    return 'unknown'

In [429]:
user_data.loc[:,'insurance_category'] = user_data.cause.apply(insurance_cause_category)
#user_data.groupby('insurance_category').count()
user_data.insurance_category.value_counts()

страхование автомобиля          876
страхование имущества           762
страхование во время отпуска    656
Name: insurance_category, dtype: int64

Категоризируем статус выплаты компенсации — 0 выплат, выплачена часть, полностью выплачена.

In [279]:
def compensation_paid_category(value):
    if value == 1:
        return 'fully'
    elif value == 0:
        return 'not paid'
    else: return 'partially'

In [280]:
user_data.loc[:,'compensation_paid'] = user_data.perc_of_compensation_paid.apply(compensation_paid_category)
#user_data.groupby('compensation_paid').id.count()
user_data.compensation_paid.value_counts()

partially    1788
fully         356
not paid      150
Name: compensation_paid, dtype: int64

Категоризируем `income`, разбив данные по квантилям.

In [430]:
user_data.loc[:,'income_quantile'] = pd.qcut(user_data['income'], q=4, precision=0)
user_data.income_quantile.value_counts()

(-1.0, 106888.0]         574
(241590.0, 2700040.0]    574
(106888.0, 168690.0]     573
(168690.0, 241590.0]     573
Name: income_quantile, dtype: int64

## Результаты

Подготовим 3 таблицы и изучим результат.

In [418]:
compensation_by_age = user_data.pivot_table(index=['compensation_paid','age_group'], values='id', aggfunc='count').transpose()
compensation_by_age

compensation_paid  fully                 not paid                 partially  \
age_group         зрелый молодой пожилой   зрелый молодой пожилой    зрелый   
id                   100      52     204       62      31      57       884   

compensation_paid                  
age_group         молодой пожилой  
id                    206     698

In [422]:
compensation_by_cause = user_data.pivot_table(index=['compensation_paid','insurance_category'], values='id', aggfunc='count').transpose()
compensation_by_cause

compensation_paid                   fully                               \
insurance_category страхование автомобиля страхование во время отпуска   
id                                    182                           78   

compensation_paid                                      not paid  \
insurance_category страхование имущества страхование автомобиля   
id                                    96                     54   

compensation_paid                                                      \
insurance_category страхование во время отпуска страхование имущества   
id                                           42                    54   

compensation_paid               partially                               \
insurance_category страхование автомобиля страхование во время отпуска   
id                                    640                          536   

compensation_paid                         
insurance_category страхование имущества  
id                                   612

In [423]:
compensation_by_income = user_data.pivot_table(index=['compensation_paid','income_quantile'], values='id', aggfunc='count').transpose()
compensation_by_income

compensation_paid            fully                                            \
income_quantile   (-1.0, 106888.0] (106888.0, 168690.0] (168690.0, 241590.0]   
id                             110                   85                   79   

compensation_paid                               not paid                       \
income_quantile   (241590.0, 2700040.0] (-1.0, 106888.0] (106888.0, 168690.0]   
id                                   82               56                   34   

compensation_paid                                                   partially  \
income_quantile   (168690.0, 241590.0] (241590.0, 2700040.0] (-1.0, 106888.0]   
id                                  32                    28              408   

compensation_paid                                            \
income_quantile   (106888.0, 168690.0] (168690.0, 241590.0]   
id                                 454                  462   

compensation_paid                        
income_quantile   (241590.0, 2700040.0]  
id                                  464

*ответьте на вопрос — есть ли зависимость между возрастной группой и статусом выплаты?*

По данным можно сделать вывод, что 
* клиентам относящие к категории "пожилой" в большем количестве выплачивается полная компенсация.
* клиентам относящие к категории "зрелый" в большем количестве не выплачивается компенсация.
* клиентам относящие к категории "зрелый" в большем количестве частично выплачивается компенсация.

*есть ли зависимость между причиной страхового случая и статусом выплаты?*

По данным можно сделать вывод, что

* больше всего полная компенсация выплачивается в случае страхование автомобиля
* больше всего компенсация не выплачивается в случае страхование автомобиля и страхование имущества.
* больше всего частично компенсация выплачивается так же в случае страхование автомобиля и страхование имущества.

*есть ли зависимость между доходом клиента и статусом выплаты?*

По данным можно сделать вывод, что

* больше всего полная компенсация выплачивается клиентам с самым низким доходом
* больше всего компенсация не выплачивается так же клиентам с самым низким доходом
* больше всего частично компенсация выплачивается клиентам с самым высоким доходом и с доходом в интервале третьего квантиля

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Всё отлично!